<a href="https://colab.research.google.com/github/Seong-jieun/Crawling/blob/main/Crawling_Kyobobook/Crawling_Kyobobook_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요

- Colab에서 Selenium, BeautifulSoup으로 크롤링하기
- 크롤링 할 페이지
    - 교보문고 > 국내도서 > 소설 > 한국소설 > 베스트셀러
    - https://product.kyobobook.co.kr/category/KOR/0101#?page=1&type=best&per=20
- 크롤링 할 데이터
    - 책코드, 제목, 작가, 책소개, 리뷰별점, 출판사
- 2025.02.27 기준
- 참고자료
    - https://sol-aftercoding.tistory.com/entry/Python-%EA%B5%90%EB%B3%B4-%EB%AC%B8%EA%B3%A0-%ED%81%AC%EB%A1%A4%EB%A7%81-%ED%95%98%EA%B8%B0-with-Selenium-BeautifulSoup?category=1098008
    - https://xl-shine.tistory.com/44

# 라이브러리 설치

In [1]:
!pip install selenium
!pip install beautifulsoup4
!pip install webdriver-manager

In [2]:
!pip install seleniumbase

In [3]:
# 버전 확인
!python --version
import selenium
print(selenium.__version__)

Python 3.11.11
4.29.0


In [4]:
import time
from seleniumbase import Driver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd

# 데이터 크롤링

In [5]:
# 크롬드라이버 경로 지정
driver_path = 'C:\Program Files (x86)\Google\chromedriver-win64\chromedriver-win64\chromedriver.exe'

In [6]:
service = Service(executable_path=driver_path)
chrome_options =  webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--blink-settings=imagesEnabled=false')

driver = Driver(browser="chrome", headless=True)
url = 'https://product.kyobobook.co.kr/category/KOR/0101#?page=1&type=best&per=20'
driver.get(url)

In [7]:
# 실행 확인
print(driver)

<selenium.webdriver.chrome.webdriver.WebDriver (session="3b38a810337fdd4dc89ceb55de0b914d")>


In [8]:
# 파싱
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [9]:
# li 태그 가져오기
items = soup.select('.prod_item')

# 크롤링 한 데이터를 담을 배열
data = []

# 각 li 태그에서 책 정보 데이터 가져오기
for item in items:

    # 책번호
    link_number = item.find('a', class_="prod_link")['href'].split("/")[-1].strip()

    # 제목
    title = item.find('span', class_="prod_name").get_text(strip=True)

    # 작가
    author = item.find('span', class_="prod_author").find('a').text.strip()

    # 책소개
    introduction = item.select_one('.prod_introduction').text.strip()

    # 리뷰
    review = item.select_one('.review_klover_text').text.strip()

    data.append((link_number, title, author, introduction, review))

- 출판사 정보 크롤링

In [15]:
# 출판사 정보 크롤링 (책제목 클릭 -> 책 상세페이지로 이동 -> 출판사 확인 -> 크롤링)
# 책 상세페이지 주소는 'base_url + link_number' 조합
base_url = "https://product.kyobobook.co.kr/detail/"

publishers = []
for link_number in [item[0] for item in data]:
    driver.get(base_url + link_number)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    publisher = soup.select_one('.btn_publish_link').text.strip()

    publishers.append(publisher)

In [14]:
publishers

[]

In [16]:
# 데이터 프레임 생성
df = pd.DataFrame(data, columns=['Number', 'Title', 'Author', 'Introduction', 'Review'])
df['Publisher'] = publishers
df

,Number,Title,Author,Introduction,Review,Publisher
0,S000001632467,모순,양귀자,양귀자 소설의 힘을 보여준 베스트셀러 『모순』. 1998년에 초판이 출간된 이후 1...,9.8,쓰다
1,S000000610612,소년이 온다,한강,"2014년 만해문학상, 2017년 이탈리아 말라파르테 문학상을 수상하고 전세계 20...",9.79,창비
2,S000200550189,급류,정대건,2020년 《한경신춘문예》에 장편소설 『GV 빌런 고태경』이 당선되어 작품 활동을 ...,9.44,민음사
3,S000000610650,채식주의자,한강,2016년 인터내셔널 부커상을 수상하며 한국문학의 입지를 한단계 확장시킨 한강의 장...,9.55,창비
4,S000000781116,작별하지 않는다,한강,2016년 『채식주의자』로 인터내셔널 부커상을 수상하고 2018년 『흰』으로 같은 ...,9.71,문학동네
5,S000215787788,그 개와 혁명(제 48회 이상문학상 작품집 2025년),예소연 외,제48회 이상문학상 대상에 예소연의 「그 개와 혁명」이 선정되었다. 1980년대 학...,9.75,다산책방
6,S000001632473,나는 소망한다 내게 금지된 것을,양귀자,1992년 초판이 나오자마자 페미니즘 논란과 함께 화제의 중심에 오른 양귀자의 장편...,9.84,쓰다
7,S000000781065,홍학의 자리,정해연,10년 가까이 스릴러 장르에 매진하며 장편 단편 할 것 없이 독자들의 눈길을 사로잡...,9.49,엘릭시르
8,S000000780090,흰,한강,한국인 최초 맨부커상 수상 작가 한강의 소설 『흰』. 2018년 맨부커 인터네셔널 ...,9.67,문학동네
9,S000000778206,희랍어 시간,한강,한국인 최초 맨부커상 수상 작가 한강의 장편소설 『희랍어 시간』. 말을 잃어가는 한...,9.67,문학동네


In [19]:
# csv 파일로 저장
df.to_csv('Kyobo bestseller_20250227.csv', index=False)

In [18]:
# 드라이버 종료
driver.close()
driver.quit()

# 회고

- 26일과 27일에 한번씩 크롤링을 진행했는데 하루 사이에 베스트 셀러 순위가 바뀐점이 흥미로웠다.

- Keep
    - 자주 사용하지 않던 vscode에서 크롤링을 시도했다.
    - 코랩에서 셀레니움을 사용하기 위해 검색과 시도를 많이 했다.
- Problem
    - 26일에는 없었지만 27일자 베스트셀러 순위에 예약판매 도서가 있어서 '예약판매'라는 라벨을 지우는 작업을 시도했지만, 다른 코드를 다 수정해야 했고 수정해도 크롤링이 제대로 되지 않았다.
    - 에러 발생 원인을 정확히 파악하지 못했다.
    - 코드를 짤 때 어떤 태그를 써야하는지 헷갈렸다.
- Try
    - 다른 동적 웹페이지 크롤링을 시도해본다.
    - vscode에서 진행한다.
    - 작성한 노트를 리뷰한다.